<a href="https://colab.research.google.com/github/itsLawrenceChao/USER/blob/master/docs/llm-practical/code/OpenAI_Agents_SDK_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OpenAI Agents SDK 多代理系統簡介

OpenAI 的 **Agents SDK** 是一套設計來簡化多代理人（multi-agent）工作流程的工具，讓開發者能夠快速建立由多個 AI 代理人協同運作的系統。

## 主要特點

- **Agents 定義**  
  您可以根據不同任務建立不同角色的代理人，例如客服專員、旅行規劃師或當地專家，每個代理人負責特定任務。

- **Handoffs（交接）**  
  系統能智能地在不同代理人之間轉交控制權。例如，當主要代理人完成初步任務後，會自動將控制權交接給更專精的代理人來補充細節。

- **Guardrails（安全檢查）**  
  內建可配置的安全檢查機制，對輸入與輸出進行驗證，確保資料正確與安全，避免因輸入錯誤而產生不預期的問題。

- **Tracing & Observability（追蹤與可觀測性）**  
  提供視覺化工具，讓開發者可以追蹤代理人的執行流程與交接路徑，有助於除錯與效能優化。

In [ ]:
!pip install openai-agents -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.9/98.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.6/128.6 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 606.1/606.1 kB 18.7 MB/s eta 0:00:00


## 註冊 OpenAI platform
雖然本教學示範使用 Groq 作為免費的模型後端，但我們依然建議你在 [OpenAI platform](https://platform.openai.com/) 建立一個專案，因為 Agents SDK 提供了 Tracing（可觀測性）功能，能將每一次代理人的互動紀錄上傳到 OpenAI 後台進行視覺化與除錯。

In [ ]:
from agents import set_tracing_export_api_key

# 如果不想上傳到 OpenAI，就保持空字串
set_tracing_export_api_key('YourOpenAIKey') # 請自行填入

## 註冊 Groq(免費LLM方案)
如果手邊沒有付費 OpenAI 的 LLM 模型 API 不妨可以使用免費的資源先玩玩看。[Groq](https://console.groq.com/login) 的使用方式與介紹可以參考另一篇文章。這一步驟主要是申請API key，因為 Agents SDK 也能整合其他業者的模型，只要有提供類似 Chat Completions 的 API 端點都能相容。除了Groq外也能使用像是 GitHub Models、Google Gemini、Ollama等。

- [Groq 使用教學](https://andy6804tw.github.io/crazyai-llm/free-llm-api-integration-resources/groq-tutorial/)

In [ ]:
GROQ_API_KEY = 'YourGroqKey' # 請自行填入

## Hello world example
在這個範例中，我們建立一個最簡單的「助理代理人」，並使用 Groq 提供的免費模型來回答問題。

In [ ]:
from agents import Agent, Runner, AsyncOpenAI, OpenAIChatCompletionsModel

# 初始化 Groq Client
groq_client = AsyncOpenAI(
    base_url = 'https://api.groq.com/openai/v1',
    api_key=GROQ_API_KEY, # 填入對應的金鑰
)

# 建立一個簡單的助理代理人
agent = Agent(
    name="Assistant",
    instructions="你是一位有用的助理，請使用繁體中文回答問題。",
    model=OpenAIChatCompletionsModel(
        model="llama3-8b-8192", # 指定使用 Groq 上的 llama3-8b-8192 模型
        openai_client=groq_client,
    )
)

In [ ]:
# 輸入提問給代理人
result = await Runner.run(agent, "用Python實現費式數列")
print(result.final_output)

費式數列(Fibonacci sequence)是一種非常有名的數學概念，它是指一個數列，其中每個元素都是前面兩個元素的和。 Python 中可以使用以下程式碼來實現費式數列：

```python
def fibonacci(n):
    if n <= 0:
        return "Input should be a positive integer."
    elif n == 1:
        return [0]
    elif n == 2:
        return [0, 1]
    else:
        fib_seq = [0, 1]
        for i in range(2, n):
            fib_seq.append(fib_seq[i-1] + fib_seq[i-2])
        return fib_seq

num = int(input("請輸入您想要計算的費式數列長度："))
print(fibonacci(num))
```

這個程式碼使用了迴圈來計算費式數列，每次迴圈都將前面兩個元素的和加到數列中。當輸入的數量大於2時-program will calculate the Fibonacci sequence up to the nth number.

你可以在命令提示符（command prompt）或Python shell中執行這個程式碼，並輸入一個正整數來計算費式數列。例如，如果你輸入5，那麼程式碼將輸出費式數列 `[0, 1, 1, 2, 3]`。


## 範例二：Handoffs（交接）範例
此範例示範如何利用 handoff 功能在多個代理人之間進行任務轉交。根據使用者的問題屬性，自動分流到「英文老師」或「歷史老師」。

In [ ]:
from agents import Agent, Runner, AsyncOpenAI, OpenAIChatCompletionsModel

# 初始化 Groq Client
groq_client = AsyncOpenAI(
    base_url = 'https://api.groq.com/openai/v1',
    api_key=GROQ_API_KEY, # 填入對應的金鑰
)

# 定義英文老師代理人，負責解答英文相關問題
english_teacher = Agent(
    name="English Tutor",
    handoff_description="負責回答英文文法、單字與閱讀理解等問題",
    instructions="你是一位專業的英文老師，請詳細解釋英文文法、單字用法，並舉例說明，幫助學生理解英文內容。並使用繁體中文回答問題。",
    model=OpenAIChatCompletionsModel(
        model="qwen-qwq-32b",
        openai_client=groq_client,
    )
)

# 定義歷史老師代理人，負責解答歷史事件與背景問題
history_teacher = Agent(
    name="History Tutor",
    handoff_description="負責回答歷史事件、人物和背景問題。",
    instructions="你是一位博學的歷史老師，請針對使用者的歷史問題提供詳盡的背景介紹、事件解釋及相關細節。並使用繁體中文回答問題。",
    model=OpenAIChatCompletionsModel(
        model="qwen-qwq-32b",
        openai_client=groq_client,
    )
)

# 定義分流代理人，根據使用者問題內容自動判斷並轉交給最合適的老師
triage_assistant = Agent(
    name="Triage Agent",
    instructions="""
請根據以下規則判斷使用者問題的屬性：
1. 若問題涉及英文文法、詞彙或閱讀理解，請轉交給「English Tutor」；
2. 若問題涉及歷史事件、人物或背景，請轉交給「History Tutor」。
請根據使用者提問內容，自動選擇最適合的代理人。
若提問非兩個領域的問題，則拒絕回答。
""",
    handoffs=[english_teacher, history_teacher],
    model=OpenAIChatCompletionsModel(
        model="qwen-qwq-32b",
        openai_client=groq_client,
    )
)

sample_question = "請問美國獨立戰爭中有哪些重要事件？"
result = await Runner.run(triage_assistant, sample_question)
print("最終回覆：", result.final_output)

最終回覆： 美國獨立戰爭（1775年〜1783年）是美國從英國殖民地擴張為獨立國家的關鍵過程。這期間有不少重大的事件。

1. 波士頓茶黨事件（1773年）：這是由美國獨立戰爭導火索之一，英國通過「茶稅法」向北美殖民地出售茶葉，引發新英格蘭地區反對。1773年12月16日反對英國課稅法士绅將倉庫內的茶葉傾倒波士頓海港，成為挑戰英國統治的第一個暴力事件。

2. 来克星頓和康科德戰役（1775年4月19日）：此次戰役標誌著美國獨立戰爭的正式開始，對抗英國紅軍的主要武裝力量新英格兰民兵成功在來克星頓和康科德地區進行了擊退红軍的戰役。

3. 費城宣言（1776年7月4日）：費城宣言标志著北美13殖民地与其殖民宗主国英国断绝政治联系並成为一个独立自主的自由複合式共和聯邦國家，宣言正式由大陸會議在費城的獨立廳簽署。

4. 塔倫頓戰役（1780年10月7日）：塔倫頓戰役是美國獨立戰爭的一場激烈戰役，大陸軍成功擊敗被認為無敵的英軍輕騎兵軍隊之一翼騎兵（British Legion），使得大陸軍軍團士氣大振。

5. 結束於約克鎮之勝利（1781年10月19日）：大陸軍在一次重要戰役中擊敗英軍，並在雲山角（Yorktown）迫使英軍投降。該戰役被視為美國獨立戰爭中的決定性戰役。

以上是在美國獨立戰爭期間的重要事件，都對戰爭的結局以及最終的獨立結果起了關鍵作用。這些過程顯示了由民兵與民選政府領軍的殖民地如何對抗強大的英國威權，成功奪取自由並成為全球首次由殖民地成功獨立成為國家之一。


## 範例三：Functions（函式工具）範例
Agents SDK 本身提供了 Responses API（需付費）讓我們可直接使用官方的 web search、file search 等工具，但若想節省成本，可以透過自定義的函式（function_tool）整合第三方 API 來實作類似功能。

以下示範一個「Web 搜尋」的功能，利用 Tavily（每月有 1000 次免費呼叫）來查詢最新的網路資訊，並將結果回傳給代理人。

In [ ]:
!pip install -qU tavily-python -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.0/91.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 19.1 MB/s eta 0:00:00


In [ ]:
from tavily import TavilyClient

TAVILY_API_KEY="YourTavilyKey"
tavily_client = TavilyClient(api_key=TAVILY_API_KEY)
response = tavily_client.search("請問今天的體育新聞?")

print(response)

{'query': '請問今天的體育新聞?', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': '體育新聞|SPORT598體育網', 'url': 'https://www.sport598.com/', 'content': '提供體育新聞、體壇動態，每日匯整最新體育賽事新聞資訊，掌握nba美國職籃、台灣籃球賽事、mlb美國職棒、中華職棒、日韓職棒、歐洲足球-英超、德甲、西甲、意甲及法甲等五大足球聯賽、網球國際賽事、f1賽車、高爾夫、羽球、桌球、排球、自行車、馬拉松、奧運、運動會等世界體壇動態，鎖定', 'score': 0.6018984, 'raw_content': None}, {'title': '體育 - 中時新聞網', 'url': 'https://www.chinatimes.com/sports/?chdtv', 'content': '體育 - 中時新聞網 ... 溜馬今天（21日）主場迎戰籃網，本只是一場平凡的例行賽，卻在末節意外爆發激烈衝突，連趕來勸架的籃網主帥費南德茲', 'score': 0.49670476, 'raw_content': None}, {'title': '體育 | 即時新聞 - 星島日報', 'url': 'https://std.stheadline.com/realtime/sport/即時-體育', 'content': '即時體育新聞報道，提供國際及本地體育賽事，以及學界體育賽事等新聞資訊，緊貼每個體壇盛事。 ... 現年27歲的華拉夫施尼打法靈活，射門頗具把握力，今季在格拉斯哥流浪射入17球，已突破個人以往的球季的新高，尤其在歐霸盃射入6球，助球隊殺入8強', 'score': 0.47986987, 'raw_content': None}, {'title': '體育新聞 - 即時 - 自由時報電子報', 'url': 'https://news.ltn.com.tw/list/breakingnews/sports', 'content': '體育新聞 - 即時 - 自由時報電子報 為達最佳瀏覽效果，建議使用 Chrome、Firefox 或 Microsoft Edge 的瀏覽器。

In [ ]:
import json

from typing_extensions import TypedDict, Any
from agents import Agent, Runner, FunctionTool, RunContextWrapper, function_tool, AsyncOpenAI, OpenAIChatCompletionsModel

@function_tool
async def tavily_search(query: str) -> dict:
    """
    使用 Tavily API 進行網路搜尋，並回傳結果。
    Args:
        query: 輸入欲搜尋的內容。
    """
    tavily_client = TavilyClient(api_key=TAVILY_API_KEY)
    print(f"Tavily search with query {query}")
    response = tavily_client.search(query)
    results = response['results']
    print(f"Search Results: {results}")

    return results

# 初始化 Groq Client
groq_client = AsyncOpenAI(
    base_url='https://api.groq.com/openai/v1',
    api_key=GROQ_API_KEY,
)

# 建立能調用 tavily_search 的代理人
tavily_search_agent = Agent(
    name="Assistant",
    instructions="使用中文回答問題。",
    tools=[tavily_search],
    model=OpenAIChatCompletionsModel(
        model="qwen-qwq-32b",
        openai_client=groq_client,
    )
)

In [ ]:
result = await Runner.run(tavily_search_agent, "請問今天的體育新聞?")
print(result.final_output)

Tavily search with query 今天的體育新聞
Search Results: [{'title': '體育新聞 | ETtoday新聞雲', 'url': 'https://www.ettoday.net/news/focus/體育/', 'content': '《ETtoday新聞雲》體育新聞最新列表 ... 中信兄弟21日官辦熱身賽4比4打和味全龍，中信外野手曾頌恩今天扛下4棒，單場貢獻4分打點，包括一發2分砲', 'score': 0.6150191, 'raw_content': None}, {'title': '體育新聞 - 即時 - 自由時報電子報', 'url': 'https://news.ltn.com.tw/list/breakingnews/sports', 'content': 'tpbl場均10.2分的雲豹羅斯繳關島最高22分 亞洲盃男籃》阿巴西砍23分 台灣隊狂勝關島40分搶下首勝 亞洲盃男籃》高柏鎧、阿巴西輪流發威!', 'score': 0.5515985, 'raw_content': None}, {'title': '體育│TVBS新聞網', 'url': 'https://news.tvbs.com.tw/sports', 'content': 'tvbs 運動提供最新的體育新聞及影音，包含棒球、籃球、網球及更多運動，為您的生活加滿元氣。', 'score': 0.52232456, 'raw_content': None}, {'title': '即時新聞 - 自由體育', 'url': 'https://sports.ltn.com.tw/breakingnews', 'content': '剛歸隊的柯瑞重摔提前傷退 主帥曝當下狀況（影音） 布朗尼得分寫生涯新高 今日nba戰績 hbl》兩場冠軍賽售票金額破百萬 4隊均分各獲約20萬 tpbl》打', 'score': 0.498902, 'raw_content': None}, {'title': '新浪体育_新浪网', 'url': 'https://sports.sina.com.cn/', 'content': '新浪体育_新浪网 新浪体育 视频 视频 中国足球 国际足球 NBA 微博 新浪体育 热门 NBA 国际足球 中国足球 综合体育 美